In [16]:
import boto3
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [71]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket = 'cloudengdfs', Key = "StockX-Data-Contest-2019-3.csv")
## grab the body of the object to get core data frame
df = pd.read_csv(obj['Body'], index_col = None)

In [72]:
df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,"$1,097",$220,9/24/16,11.0,California
1,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,$685,$220,11/23/16,11.0,California
2,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,$690,$220,11/23/16,11.0,California
3,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,"$1,075",$220,11/23/16,11.5,Kentucky
4,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,$828,$220,2/11/17,11.0,Rhode Island


In [73]:
df['Brand'] = df['Brand'].str.replace('Off-White', 'nike')
df['Brand'] = df['Brand'].str.replace(' Yeezy', 'adidas')
df["Sale Price"] = df["Sale Price"].str.replace("$", "").str.replace(",", "")
df["Retail Price"] = df["Retail Price"].str.replace("$", "").str.replace(",", "")
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Release Date"] = pd.to_datetime(df["Release Date"])
df[["Sale Price", "Retail Price"]] = df[["Sale Price", "Retail Price"]].astype(int)

In [74]:
bucket_name = 'cloudengdfs'
key_name = 'original_cleaned_data.csv'
    
final_df = df.to_csv(index=False).encode('utf-8')
s3.put_object(Bucket=bucket_name, Key=key_name, Body=final_df)

{'ResponseMetadata': {'RequestId': 'FQF1983AXPJ328CN',
  'HostId': 'GmwDIC6fK3hiP+qXitQvwwLci6q+/CJuXm4SXMyKCw8Tyb27uHWLiDuFEh49BANx6MrV+SnQxl0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GmwDIC6fK3hiP+qXitQvwwLci6q+/CJuXm4SXMyKCw8Tyb27uHWLiDuFEh49BANx6MrV+SnQxl0=',
   'x-amz-request-id': 'FQF1983AXPJ328CN',
   'date': 'Tue, 16 May 2023 20:28:40 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"5987ec36a8c63b19a8420013d872c93d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5987ec36a8c63b19a8420013d872c93d"',
 'ServerSideEncryption': 'AES256'}

In [132]:
obj2 = s3.get_object(Bucket = 'cloudengdfs', Key = "original_cleaned_data.csv")
## grab the body of the object to get core data frame
df1 = pd.read_csv(obj2['Body'], index_col = None)

In [133]:
df1["Order Date"] = pd.to_datetime(df1["Order Date"])
df1["Release Date"] = pd.to_datetime(df1["Release Date"])
df1 = pd.get_dummies(df1, columns=['Brand', 'Sneaker Name'])
st = df1.iloc[:, 6:].astype(int)
df1 = df1.iloc[:, 0:6]
df1 = pd.concat([df1, st], axis = 1)

df1['markup'] = (df1['Sale Price'] - df1['Retail Price']) / df1['Retail Price']
mean_resale_price_by_model = df.groupby('Sneaker Name')['Sale Price'].mean()
df1['mean_resale_price_by_model'] = df['Sneaker Name'].map(mean_resale_price_by_model)
df1['age_in_days'] = (df1['Order Date'] - df1['Release Date']).dt.days
df1.drop(['Order Date', 'Release Date'], axis=1, inplace=True)

df1.head()


,Sale Price,Retail Price,Shoe Size,Buyer Region,Brand_adidas,Brand_nike,Sneaker Name_Adidas-Yeezy-Boost-350-Low-Moonrock,Sneaker Name_Adidas-Yeezy-Boost-350-Low-Oxford-Tan,Sneaker Name_Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015,Sneaker Name_Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,...,Sneaker Name_Nike-Zoom-Fly-Mercurial-Off-White-Total-Orange,Sneaker Name_Nike-Zoom-Fly-Off-White,Sneaker Name_Nike-Zoom-Fly-Off-White-Black-Silver,Sneaker Name_Nike-Zoom-Fly-Off-White-Pink,Sneaker Name_adidas-Yeezy-Boost-350-V2-Butter,Sneaker Name_adidas-Yeezy-Boost-350-V2-Static,Sneaker Name_adidas-Yeezy-Boost-350-V2-Static-Reflective,markup,mean_resale_price_by_model,age_in_days
0,1097,220,11.0,California,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3.986364,788.704819,342
1,685,220,11.0,California,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.113636,656.631111,282
2,690,220,11.0,California,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.136364,622.801020,282
3,1075,220,11.5,Kentucky,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3.886364,937.754967,282
4,828,220,11.0,Rhode Island,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.763636,804.304438,202


In [134]:
bucket_name = 'cloudengdfs'
key_name = 'feature_eng_data.csv'
    
final_df = df1.to_csv(index=False).encode('utf-8')
s3.put_object(Bucket=bucket_name, Key=key_name, Body=final_df)

{'ResponseMetadata': {'RequestId': '9YZA16EJSHSD62AH',
  'HostId': 'exaUjBnQxiCCOUt9zZ9e8ih3VOn2MpIRGszXeQxEl2BtJgLuzxCN2n0sqdbsPGi7fruNlKYEOMM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'exaUjBnQxiCCOUt9zZ9e8ih3VOn2MpIRGszXeQxEl2BtJgLuzxCN2n0sqdbsPGi7fruNlKYEOMM=',
   'x-amz-request-id': '9YZA16EJSHSD62AH',
   'date': 'Wed, 17 May 2023 01:08:47 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"2f5d430b3e2c5a21e53b7848ecdbce13"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"2f5d430b3e2c5a21e53b7848ecdbce13"',
 'ServerSideEncryption': 'AES256'}

In [135]:
df1.shape

(99956, 59)